In [10]:
!pip install evaluate

In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
import evaluate
import torch

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/python-codes-time-complexity/test_set.csv
/kaggle/input/python-codes-time-complexity/train_set.csv


# Dataset uploading

In [12]:
train_set = load_dataset("csv", data_files="/kaggle/input/python-codes-time-complexity/train_set.csv")
test_set = load_dataset("csv", data_files="/kaggle/input/python-codes-time-complexity/test_set.csv")

# Tokenizing data

In [13]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

def tokenize_data(samples):
    return tokenizer(samples['code'], padding='max_length', truncation=True)

train_set = train_set.map(tokenize_data, batched=True)
eval_set = test_set.map(tokenize_data, batched=True)

Map:   0%|          | 0/978 [00:00<?, ? examples/s]

# Model

In [14]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Hyperparameters

In [80]:
training_args = TrainingArguments(output_dir="training_results", eval_strategy="epoch", fp16=True)

# Evaluating

In [81]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Trainer

In [82]:
trainer = Trainer(
    model=model, 
    args=training_args,
    train_dataset=train_set,
    eval_dataset=eval_set,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>